In [38]:
import os
import pandas as pd
from keras.layers import Rescaling  # Camada de escalonamento
from keras.optimizers import Adam  # optimizador Adam
# Classe utilizada para acompanhamento durante o treinamento onde definimos os atributos que serão considerados para avaliação
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.data import AUTOTUNE
# Função que carrega o dataset de um diretório
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.metrics import Recall, Precision
import utils

In [39]:
train_path = "./../data/fer2013/train/"
validation_path = "./../data/fer2013/test/"
models_path = "./models"

width = 48  
height = 48  
depth = 1 

epochs = 25
init_lr = 1e-3
batch_size = 32

color_mode = "grayscale"

input_shape = (height, width, depth)
save_model = os.path.join(
        models_path, "lenet-{epoch:02d}-{accuracy:.3f}-{val_accuracy:.3f}.model")

In [40]:
train_ds = image_dataset_from_directory(
        train_path,
        seed=123,
        label_mode='categorical',
        validation_split=0.3,
        subset="training",
        color_mode=color_mode,
        image_size=(height, width),
        batch_size=batch_size
    )

val_ds = image_dataset_from_directory(
    validation_path,
    seed=123,
    label_mode='categorical',
    validation_split=0.3,
    subset="validation",
    color_mode=color_mode,
    image_size=(height, width),
    batch_size=batch_size
) 

Found 28709 files belonging to 7 classes.
Using 20097 files for training.
Found 7178 files belonging to 7 classes.
Using 2153 files for validation.


In [41]:
rescaling_layer = Rescaling(1./255) ## normalizando os valores dos pixels
# pré-busca em buffer para que se possa produzir dados do disco sem que a E/S se torne um bloqueio
train_ds = train_ds.map(lambda x, y: (rescaling_layer(x), y), 
                        num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (rescaling_layer(x), y),
                    num_parallel_calls=AUTOTUNE)


In [42]:
model = utils.create_model(input_shape=input_shape)

In [43]:
opt = Adam(lr=init_lr, decay=init_lr / epochs)

model.compile(loss="categorical_crossentropy",
                  optimizer=opt, metrics=["accuracy", Precision()])

/home/elisaveloso/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [44]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_2 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 24, 24, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 activation_6 (Activation)   (None, 24, 24, 64)       

In [45]:
checkpoint1 = ModelCheckpoint(
    save_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(
    save_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1, checkpoint2]

In [46]:
H = model.fit(train_ds,
                  validation_data=val_ds,
                  epochs=epochs,
                  verbose=1,
                  callbacks=callbacks_list
                  )



Epoch 1/25
628/629 [============================>.] - ETA: 0s - loss: 1.6589 - accuracy: 0.3356 - precision_3: 0.6828
Epoch 1: val_loss improved from inf to 1.49767, saving model to ./models/lenet-01-0.336-0.425.model


INFO:tensorflow:Assets written to: ./models/lenet-01-0.336-0.425.model/assets


INFO:tensorflow:Assets written to: ./models/lenet-01-0.336-0.425.model/assets



Epoch 1: val_accuracy improved from -inf to 0.42499, saving model to ./models/lenet-01-0.336-0.425.model


INFO:tensorflow:Assets written to: ./models/lenet-01-0.336-0.425.model/assets


...

INFO:tensorflow:Assets written to: ./models/lenet-04-0.549-0.507.model/assets


629/629 [==============================] - 48s 76ms/step - loss: 1.1872 - accuracy: 0.5495 - precision_3: 0.7508 - val_loss: 1.2806 - val_accuracy: 0.5067 - val_precision_3: 0.7158
Epoch 5/25
628/629 [============================>.] - ETA: 0s - loss: 1.0737 - accuracy: 0.5932 - precision_3: 0.7639
Epoch 5: val_loss did not improve from 1.28057

Epoch 5: val_accuracy improved from 0.50673 to 0.53553, saving model to ./models/lenet-05-0.593-0.536.model


INFO:tensorflow:Assets written to: ./models/lenet-05-0.593-0.536.model/assets


INFO:tensorflow:Assets written to: ./models/lenet-05-0.593-0.536.model/assets


629/629 [==============================] - 47s 75ms/step - loss: 1.0737 - accuracy: 0.5932 - precision_3: 0.7639 - val_loss: 1.3014 - val_accuracy: 0.5355 - val_precision_3: 0.6827
Epoch 6/25
628/629 [============================>.] - ETA: 0s - loss: 0.9566 - accuracy: 0.6418 - precision_3: 0.7812
Epoch 6: val_loss did not improve from 1.28057

Epoch 6: val_accuracy did not improve from 0.53553
629/629 [==============================] - 58s 91ms/step - loss: 0.9567 - accuracy: 0.6418 - precision_3: 0.7812 - val_loss: 1.4413 - val_accuracy: 0.5123 - val_precision_3: 0.6241
Epoch 7/25
628/629 [============================>.] - ETA: 0s - loss: 0.8398 - accuracy: 0.6889 - precision_3: 0.7996
Epoch 7: val_loss did not improve from 1.28057

Epoch 7: val_accuracy did not improve from 0.53553
629/629 [==============================] - 47s 74ms/step - loss: 0.8399 - accuracy: 0.6889 - precision_3: 0.7996 - val_loss: 1.6132 - val_accuracy: 0.5081 - val_precision_3: 0.5833
Epoch 8/25
628/629 [===

In [ ]:
report = H.history

In [ ]:
report = pd.DataFrame(report)

In [ ]:
report.to_csv("report-5.csv")